In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import logging

from conda_forge_tick.utils import setup_logging

setup_logging("INFO")

## Find Some Files

In [3]:
from conda_forge_tick.lazy_json_backends import get_sharded_path

In [4]:
get_sharded_path("node_attrs/ucx-split.json")

'node_attrs/d/c/d/6/1/ucx-split.json'

In [12]:
from conda_forge_tick.lazy_json_backends import LazyJson

with LazyJson("node_attrs/ucx-split.json") as attrs:
    print(attrs.data.get("version", None))

None


In [13]:
from conda_forge_tick.feedstock_parser import load_feedstock_local



attrs = {}
load_feedstock_local("ucx-split", attrs)

bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell


{'feedstock_name': 'ucx-split',
 'parsing_error': False,
 'branch': 'main',
 'raw_meta_yaml': '{% set version = "1.16.0" %}\n\npackage:\n  name: ucx-split\n\nsource:\n  url: https://github.com/openucx/ucx/archive/v{{ version }}.tar.gz\n  sha256: bf1c1ce6583ea1ab805feceb000157dc1fb825a60ddac0fbd9a329a010f9ad72\n\nbuild:\n  skip: true  # [not linux or cuda_compiler_version == "None"]\n  number: 3\n\noutputs:\n  - name: ucx\n    version: {{ version }}\n    build:\n      run_exports:\n        - {{ pin_subpackage("ucx", max_pin="x.x") }}\n      ignore_run_exports_from:\n        - {{ compiler("cuda") }}\n        - cuda-cudart-dev          # [(cuda_compiler_version or "None").startswith("12")]\n      missing_dso_whitelist:\n        - "*libcuda.so*"\n        - "*libcudart.so*"\n        - "*libnvidia-ml.so*"\n    requirements:\n      build:\n        - {{ stdlib("c") }}\n        - {{ compiler("c") }}\n        - {{ compiler("cxx") }}\n        - {{ compiler("cuda") }}\n        - automake\n        

In [14]:
attrs["version"]

KeyError: 'version'

## Load the graph

In [7]:
from conda_forge_tick.all_feedstocks import get_all_feedstocks
from conda_forge_tick.utils import load_graph

gx = load_graph()

In [ ]:
import pprint
import yaml

node = "setuptools_scm"

print(yaml.dump(gx.nodes[node]["payload"].data["requirements"]))

## Check Some Version Stuff

In [ ]:
from conda_forge_tick.update_upstream_versions import update_upstream_versions

In [ ]:
import copy

gxc = copy.deepcopy(gx)

for node in list(gxc.nodes):
    if node not in ["tzdata", "openssl", "jpeg", "cddlib"]:
        gxc.remove_node(node)

In [ ]:
update_upstream_versions(gxc, debug=True)

## Look at a Migration

In [ ]:
from conda_forge_tick.auto_tick import migration_factory, add_arch_migrate

In [ ]:
mgs = []
add_arch_migrate(mgs, gx)

In [ ]:
mg = mgs[0]

In [ ]:
node_to_check = "maturin"

In [ ]:
from conda_forge_tick.make_graph import get_deps_from_outputs_lut
import networkx as nx

arches = {
    "linux_aarch64": "default",
    "linux_ppc64le": "default",
}


graph2 = nx.create_empty_copy(gx)
for node, attrs in gx.nodes(data="payload"):
    for plat_arch in arches:
        deps = set().union(
            *attrs.get(
                f"{plat_arch}_requirements",
                attrs.get("requirements", {}),
            ).values()
        )
        for dep in get_deps_from_outputs_lut(deps, gx.graph["outputs_lut"]):
            graph2.add_edge(dep, node)

In [ ]:
print(sorted(graph2.predecessors(node_to_check)))

In [ ]:
from conda_forge_tick.utils import pluck

graph = graph2
packages = mg.target_packages.copy()
for target in mg.target_packages:
    if target in graph.nodes:
        packages.update(nx.ancestors(graph, target))
for node in list(graph.nodes.keys()):
    if node not in packages:
        pluck(graph, node)
# post-plucking cleanup
graph.remove_edges_from(nx.selfloop_edges(graph))

In [ ]:
print(sorted(graph2.predecessors(node_to_check)))

In [ ]:
from conda_forge_tick.migrators.migration_yaml import all_noarch

ignored_packages = {
    "make",
    "perl",
    "toolchain",
    "posix",
    "patchelf",  # weird issue
}

latest_preds = sorted(graph.predecessors(node_to_check))

for node, attrs in list(graph.nodes("payload")):
    if (
        (not attrs)
        or node.endswith("_stub")
        or node.startswith("m2-")
        or node.startswith("m2w64-")
        or node.startswith("__")
        or (node in ignored_packages)
        or all_noarch(attrs)
    ):
        pluck(graph, node)
        new_latest_preds = sorted(graph.predecessors(node_to_check))
        if new_latest_preds != latest_preds:
            print(node)
            print("    new preds:", new_latest_preds)
            print("    old preds:", latest_preds)
            print(" ")
            latest_preds = new_latest_preds

# post-plucking cleanup
graph.remove_edges_from(nx.selfloop_edges(graph))

In [ ]:
print(sorted(graph2.predecessors(node_to_check)))

In [ ]:
import os
from conda_forge_tick.contexts import MigratorContext, MigratorSessionContext


mctx = MigratorSessionContext(
    circle_build_url=os.getenv("CIRCLE_BUILD_URL", ""),
    graph=gx,
    smithy_version="",
    pinning_version="",
    github_username="",
    github_password="",
    github_token="",
    dry_run=False,
)

mmctx = MigratorContext(session=mctx, migrator=mg)
mg.bind_to_ctx(mmctx)

In [ ]:
mg.graph["outputs_lut"]

In [ ]:
import networkx as nx

In [ ]:
print(sorted(mg.graph.predecessors("bcrypt")))

In [ ]:
mgs = []
add_rebuild_broken_migrator(mgs, gx)

In [ ]:
mg = mgs[0]

In [ ]:
import os
from conda_forge_tick.contexts import MigratorContext, MigratorSessionContext


mctx = MigratorSessionContext(
    circle_build_url=os.getenv("CIRCLE_BUILD_URL", ""),
    graph=gx,
    smithy_version="",
    pinning_version="",
    github_username="",
    github_password="",
    github_token="",
    dry_run=False,
)

mmctx = MigratorContext(session=mctx, migrator=mg)
mg.bind_to_ctx(mmctx)

In [ ]:
mmctx.effective_graph.nodes

In [ ]:
mg_name = "libffi33"

mgs = []
migration_factory(mgs, gx, only_keep=[mg_name])

for i in range(len(mgs)):
    if mgs[i].name == mg_name:
        break
mg = mgs[i]

In [ ]:
import copy

attrs = copy.deepcopy(mg.graph.nodes["python"]["payload"].data)

attrs["branch"] = "3.6"
mg.filter(attrs)

In [ ]:
import os
from conda_forge_tick.contexts import MigratorContext, MigratorSessionContext


mctx = MigratorSessionContext(
    circle_build_url=os.getenv("CIRCLE_BUILD_URL", ""),
    graph=gx,
    smithy_version="",
    pinning_version="",
    github_username="",
    github_password="",
    github_token="",
    dry_run=False,
)

mmctx = MigratorContext(session=mctx, migrator=mg)
mg.bind_to_ctx(mmctx)


In [ ]:
mmctx.effective_graph.nodes

## Check the Status Report

In [ ]:
from conda_forge_tick.status_report import  graph_migrator_status

In [ ]:
out2, build_sequence, gv = graph_migrator_status(mg, mg.graph)

In [ ]:
gv.view()

In [ ]:
build_sequence

In [ ]:
len(build_sequence)

In [ ]:
gx.nodes["proj"]["payload"].data

## Debug Pinning Migrations

In [8]:
from conda_forge_tick.auto_tick import create_migration_yaml_creator

In [9]:
mgs = []
create_migration_yaml_creator(mgs, gx, pin_to_debug="ucx")

pinning migrations


2024-05-03 12:50:30,671 INFO     conda_forge_tick.auto_tick || pinning_name|package_name|feedstock_name: ucx|ucx|ucx-split


In [10]:
gx.nodes["ucx-split"]["payload"].get("version")